In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from models import Model, Trainer
from env_test import Env
from mcts import MCTS

class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

model =Model()# LSTMTagger()#Model()
env = Env(1,1)

args = dotdict({'cpuct':0.5, 'iters':1000})
mcts2 = MCTS(env, model, args)
#val = list(mcts2.sampling())
#print(len(val))
#val = [v for v in val if v.parent and v.parent.times_visited > 2]
#print(len(val))
t = Trainer(env, batch_size=50)

In [1]:
import torch
from torch import Tensor, nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

def get_batch(nodes_buc, batch_size=20):
    """remove all nodes that have not history"""
    n = len(nodes_buc)
    index = [
        np.random.randint(0, n - 1)
        for _ in range(batch_size)
    ]
    #print(len(self.replay), index, [self.replay[i] for i in index]   )
    return index#[nodes_buc[i] for i in index]


def train_model(Xa,yr,yp, model, batch_size=10, net_iters=200):
    optimizer = optim.SGD(model.parameters(), lr=0.1)#, momentum=0.03, weight_decay=0.1)
    loss_backet = []
    for i in range(net_iters):
        ind = get_batch(Xa)

        X, real_reward, real_prob = Xa[ind], yr[ind], yp[ind]#self.transform_bach_as_input(batch, model)
        model.train()
        optimizer.zero_grad()
        #print(X)
        p_pred, v_pred = model(X)
        # print('pr  ', probability, 'pp  ', p_pred)
        val_loss = torch.mean((Variable(Tensor(real_reward)) - v_pred) ** 2)  # , Variable(Tensor([10]))
        loss = val_loss - torch.mean(Variable(Tensor(real_prob)) * torch.log(p_pred))
        loss.backward()
        optimizer.step()
        loss_backet.append(loss.data.numpy())
        
    return loss_backet

In [12]:
def get_linear_out(samples):
    X, yv, yp, cresult = t.transform_bach_as_input(samples, model)
    return X, yv, yp

def env_high_dificult(Env, agent, Trainer):
    
    epoch = iter_lim = 10
    stat = {}
    ind_dificult = 1
    env = Env(1, ind_dificult)
    while epoch:
        epoch -= 1
        print(stat, len(mcts2.Nodes))
        if agent.sum_reward > 10:
            stat[ind_dificult] = [iter_lim - epoch, agent.sum_reward]
            agent.sum_reward = 0
            ind_dificult += 1
            epoch = iter_lim
            env = Env(1, ind_dificult)
            agent.env = env
            print('new env', agent.sum_reward)
        
        val = list(agent.sampling())
        val = Trainer.clean_unpredict_node(val)
        X, yv, yp = get_linear_out(val)
        lossb = train_model(X, yv, yp, agent.model, net_iters=500)
        #plt.plot(lossb)
        #plt.show()
        
    return stat

mcts2 = MCTS(env, model, args)
env_high_dificult(Env, mcts2, t)

{} 1
{} 1799
new env 0
{1: [2, 64]} 3417
{1: [2, 64]} 3862
{1: [2, 64]} 4137
{1: [2, 64]} 4359
{1: [2, 64]} 4515
{1: [2, 64]} 4625
{1: [2, 64]} 4787
{1: [2, 64]} 4887
{1: [2, 64]} 4982
{1: [2, 64]} 5061


{1: [2, 64]}

In [32]:
class GeneralActionModel(nn.Module):
    """
        predict general action:
          ChoseLetter
          SeeDif
          CreateObj
          EditObj
          ExpectObj
    """
    
    def __init__(self, env, task, vocab_size, args, internal_mes_size=32, hidden=64):
        super(GeneralActionModel, self).__init__()
        self.env = env
        mcts = MCTS(env, self, args)
        self.task = Variable(Tensor(list(range(vocab_size)) + task))
        
        self.inp_layer = nn.Linear(self.task.shape[0], hidden)
        #self.internal = nn.RNNCell(internal_mes, vocab_size + task_size)
        self.internal = nn.Linear(internal_mes_size, self.task.shape[0])
        self.act = nn.Linear(hidden, 5)
        self.message = nn.Linear(hidden, internal_mes_size)
        
    def forward(self, x):
        mes = self.internal(x)
        
        x = mes * self.task #/ torch.sqrt(torch.max(1 + torch.sum(mes)))
        #print(x, mes * self.task, torch.sqrt(1 + torch.sum(mes)))
        x = F.relu(self.inp_layer(x))
        act_prob = F.softmax(self.act(x))
        out_mes = F.tanh(self.message(x))
        return act_prob, out_mes
        
        
    
    
class ChoseLetter(nn.Module):
    def __init__(self, env, internal_mes_size=32, hidden=64, ):
        super(ChoseLetter, self).__init__()
        self.chl_model = Model()
        mcts = MCTS(env, self.chl_model, args)
        
        self.inp_layer = nn.Linear(internal_mes_size, hidden)
        
        
    def get_action(self):
        return a, sup_vec, back_vec
    

    

In [36]:
args = dotdict({'cpuct':0.5, 'iters':1000})
model = GeneralActionModel(env=HigerEnv(), task=[1,1], vocab_size=8, internal_mes_size=32, args=args)
model(torch.randn([2,32]))

/home/denis/anaconda3/envs/tourch_gym/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


(tensor([[0.1845, 0.2545, 0.1178, 0.2528, 0.1904],
         [0.1539, 0.3197, 0.1697, 0.1890, 0.1677]], grad_fn=<SoftmaxBackward>),
 tensor([[-0.0843, -0.3319,  0.0319, -0.1858,  0.3311, -0.2201, -0.0689,  0.0198,
          -0.2517, -0.2049,  0.0265,  0.1109, -0.3542,  0.1263, -0.0981,  0.1656,
          -0.3662, -0.0779,  0.0835,  0.0142, -0.2712, -0.3484, -0.0005, -0.3935,
          -0.3414, -0.1377, -0.4148, -0.1733, -0.0391, -0.2754, -0.0613, -0.0202],
         [-0.0980, -0.2806,  0.0192,  0.0239,  0.2604, -0.1483,  0.0228,  0.3852,
          -0.2252, -0.0396, -0.1163,  0.1580,  0.1632,  0.0867,  0.2128,  0.0624,
          -0.3019, -0.0999, -0.1019, -0.1456, -0.1890, -0.3970,  0.1320, -0.1090,
          -0.2721, -0.2288, -0.2160, -0.1942,  0.1088, -0.2711,  0.2129, -0.3104]],
        grad_fn=<TanhBackward>))

In [35]:
class HigerEnv:
    def __init__(self):
        models = []
        self.actions = range(len(models))
    
    def do_move(self, a, mes):
        return models[a].get_action(mes) # immediate_reward, out_mes

In [ ]:
# So